In [ ]:
import numpy as np
import csv
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed
from keras.layers import Conv2D, Dense, Dropout, Activation, Flatten, MaxPooling2D, LSTM
from keras.optimizers import SGD

In [ ]:
SET50 = ['ADVANC', 'AOT', 'BANPU', 'BBL', 'BCP', 'BDMS',
        'BEM', 'BH', 'BJC', 'BTS', 'CENTEL', 'CPALL', 
         'CPF', 'CPN', 'DTAC', 'EGCO', 'HMPRO', 
         'INTUCH', 'IRPC', 'KBANK', 'KCE', 'KKP', 'KTB', 'LH', 'MINT', 'PTT',
        'PTTEP', 'ROBINS', 'SCB', 'SCC', 
        'TCAP', 'TISCO', 'TMB', 'TOP', 'TRUE', 'TU']

In [ ]:
lr = 0.1
epochs = 10000
batch_size = 100
optimizer = SGD(lr=lr)
activator = 'tanh'
loss_method = 'mse'
metrics = ['mae', 'accuracy']

In [ ]:
def load_stock_data(symbol, year_start, year_end):
    stock_data = []
    with open('../Data set/FIXED_SET50/' + symbol + '.BK.csv', 'r') as csv_file:
        file_data = csv.reader(csv_file, delimiter=',')
        file_data = list(file_data)[1:]
        moving_average_data = []
        for row in file_data:
            if row[1] is '':
                continue
            elif len(moving_average_data) < 30:
                moving_average_data.append(float(row[1]))
            elif int(row[0][0:4]) >= year_start and int(row[0][0:4]) <= year_end:
                ma = np.sum(moving_average_data) / 30
                temp = float(row[1])
                diff_ma = temp - ma
                stock_data.append([diff_ma])
                del moving_average_data[0]
                moving_average_data.append(float(row[1]))
    return stock_data

In [ ]:
stock_datas = [ load_stock_data(x, 2008, 2017) for x in SET50 ]

min_count = len(stock_datas[0])
rounds = min_count - 30
train_index = int(rounds * 0.7)

In [ ]:
def create_cnn():
    model = Sequential()
    model.add(Conv2D(64, (7, 1), input_shape=(len(stock_datas), 30, 1), padding='same'))
    model.add(Activation(activator))
    model.add(MaxPooling2D(pool_size = (2, 1), data_format= "channels_last"))
    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation(activator))
    model.add(Dense(256))
    model.add(Activation(activator))
    model.add(Dropout(0.2))
    model.add(Dense(len(stock_datas)))
    return model

def create_model():
    cnn = create_cnn()
    model = Sequential()
    model.add(TimeDistributed(cnn, input_shape=(1, len(stock_datas), 30, 1)))
    model.add(LSTM(512, return_sequences=True))
    model.add(Activation(activator))
    model.add(Dropout(0.2))
    model.add(LSTM(512, return_sequences=True))
    model.add(Activation(activator))
    model.add(Dropout(0.2))
    model.add(LSTM(512, return_sequences=True))
    model.add(Activation(activator))
    model.add(Dropout(0.2))
    model.add(Dense(len(stock_datas)))
    model.summary()
    return model

In [ ]:
model = create_model()
model.compile(loss=loss_method, optimizer=optimizer, metrics=metrics)

In [ ]:
data = []
count = 0
index = 0
while index < rounds:
    temp = [i[index : index + 30] for i in stock_datas]
    data.append([temp])
    index += 1
    
x = np.asarray(data)
x_train = x[0: train_index].astype('float32')
x_test = x[train_index: train_index + 1].astype('float32')

print(x_train.shape, x_test.shape)

In [ ]:
target = []
index = 0
while index < rounds:
    temp = [stock_datas[i][index + 30][0] for i in range(len(stock_datas))]
    target.append([temp])
    index += 1
    
y = np.asarray(target)
y_train = y[0: train_index].astype('float32')
y_test = y[train_index: rounds].astype('float32')

print(y_train.shape, y_test.shape)

In [ ]:
model.fit(x_train, y_train, epochs = epochs, batch_size = batch_size, verbose= 1)
model.save('cnnlstmMA1Input.h5')